In [14]:
import os
import json
import requests

In [15]:
MOVIELENS_HOST = os.environ.get("MOVIELENS_HOST", "localhost")
MOVIELENS_SCHEMA = os.environ.get("MOVIELENS_SCHEMA", "http")
MOVIELENS_PORT = os.environ.get("MOVIELENS_PORT", "5000")

MOVIELENS_USER = os.environ.get("MOVIELENS_USER", "airflow")
MOVIELENS_PASSWORD = os.environ.get("MOVIELENS_PASSWORD", "airflow")

In [16]:
def _get_session():
    session = requests.Session()
    session.auth = (MOVIELENS_USER, MOVIELENS_PASSWORD)  # Replace with your credentials

    base_url = f"{MOVIELENS_SCHEMA}://{MOVIELENS_HOST}:{MOVIELENS_PORT}"

    return session, base_url

In [17]:
session, base_url = _get_session()

In [19]:
base_url

'http://localhost:5000'

In [20]:
session.get(base_url + "/ratings").json()

{'limit': 100,
 'offset': 0,
 'result': [{'movieId': 196997,
   'rating': 4.0,
   'timestamp': 1546301166,
   'userId': 27667},
  {'movieId': 1213, 'rating': 5.0, 'timestamp': 1546301290, 'userId': 27667},
  {'movieId': 55820, 'rating': 4.0, 'timestamp': 1546301706, 'userId': 27667},
  {'movieId': 2329, 'rating': 4.0, 'timestamp': 1546301788, 'userId': 27667},
  {'movieId': 4878, 'rating': 4.0, 'timestamp': 1546301842, 'userId': 27667},
  {'movieId': 589, 'rating': 4.0, 'timestamp': 1546302155, 'userId': 128817},
  {'movieId': 119145, 'rating': 2.0, 'timestamp': 1546302225, 'userId': 27667},
  {'movieId': 2716, 'rating': 4.0, 'timestamp': 1546302513, 'userId': 27667},
  {'movieId': 165, 'rating': 4.0, 'timestamp': 1546302548, 'userId': 27667},
  {'movieId': 57669, 'rating': 4.0, 'timestamp': 1546302649, 'userId': 27667},
  {'movieId': 5418, 'rating': 4.0, 'timestamp': 1546302687, 'userId': 27667},
  {'movieId': 377, 'rating': 4.5, 'timestamp': 1546302763, 'userId': 27667},
  {'movieId'